### Nonlinear Option Pricing, Master 2 Probabilités et Finance, Sorbonne Université and École polytechnique
# Homework II (optional) - Nathan FLEURY

### Due Date: 11:55 PM Sunday, February 1, 2026
You should turn in the notebook on the Moodle course website.

Please comment your code properly.

Before you turn in the notebook, press the "Run all cells" button in the toolbar, and make sure all the calculation results and graphs are produced correctly in a reasonable time frame, and then save the notebook.

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm

# Dual Problem and Upper Bounds for American Options

<strong>Primal Problem</strong> (maximizing over stopping times)

\begin{equation*}
V_t=\sup_{\tau\in\mathcal{T}_{t, T}}\mathbb{E}^\mathbb{Q}\left[D_{t, \tau}F_{\tau}\big|\mathcal{F}_t\right]
\end{equation*}

<strong>Dual Problem</strong> (minimizing over martingales) [Rogers 2002; Haugh and Kogan 2004]

\begin{equation*}
V_t=\inf_{M\in\mathcal{M}_{t, 0}}\mathbb{E}^\mathbb{Q}\left[\sup_{t\leq s\leq T}\left(D_{t, s}F_s-M_s\right)\bigg|\mathcal{F}_t\right].
\end{equation*}

where $\mathcal{M}_{t, 0}$ denotes the set of all right-continuous martingales $(M_s, s\in[t, T])$ with $M_t=0$.
The optimal martingale $M^{\ast}$ is the martingale part of the Doob-Meyer decomposition of $(S_s-S_t)/D_{0, t}$, $t\leq s\leq T$, where $S$ is the Snell envelope of the discounted payoff, i.e., $S_t=\sup_{\tau\in\mathcal{T}_{t,T}}\mathbb{E}^{\mathbb{Q}}\left[D_{0,\tau}F_{\tau}\right]$.

In particular, any martingale $M_s$ with $M_0=0$ gives an upper bound for the price at time 0, $V_0$:

$$V_0 \leq\mathbb{E}^{\mathbb{Q}}\left[\sup_{0\leq s\leq T}\left(D_sF_s-M_s\right)\right].$$

<b style="color:darkorange">Question 1.</b> Consider pricing a one-year Bermudan put option with monthly exercise, where the asset price process is assumed to follow geometric Brownian motion with $S_0=100$, $\sigma=0.2$, $r=0.1$, $q=0.02$, $K=100$ and exercise dates $t_1=\frac{1}{12}$, $t_2=\frac{2}{12}$, $\cdots$, $t_{12}=1$.

Simulate 100,000 paths and use the following martingale to find a upper bound.

(a). $M_t\equiv0$.

(b). $M_t$ is the discounted European put price with the same final maturity less the initial price.

For your reference, the price of the option is $5.152$.

In [2]:
def blackscholes_mc(S=100, vol=0.2, r=0, q=0, ts=np.linspace(0, 1, 13), npaths=10):
    """Generate Monte-Carlo paths in Black-Scholes model.

    Parameters
    ----------
    S: scalar
        The spot price of the underlying security.
    vol: scalar
        The implied Black-Scholes volatility.
    r: scalar
        The annualized risk-free interest rate, continuously compounded.
    q: scalar
        The annualized continuous dividend yield.
    ts: array_like
        The time steps of the simualtion
    npaths: int
        the number of paths to simulate

    Returns
    -------
    paths: ndarray
        The Monte-Carlo paths.
    """
    nsteps = len(ts) - 1
    ts = np.asarray(ts)[:, np.newaxis]
    W = np.cumsum(np.vstack((np.zeros((1, npaths), dtype=float),
                             np.random.randn(nsteps, npaths) * np.sqrt(np.diff(ts, axis=0)))),
                  axis=0)
    paths = np.exp(-0.5*vol**2*ts + vol*W)*S*np.exp((r-q)*ts)
    return paths


def blackscholes_price(K, T, S, vol, r=0, q=0, callput='call'):
    """Compute the call/put option price in the Black-Scholes model
    
    Parameters
    ----------
    K: scalar or array_like
        The strike of the option.
    T: scalar or array_like
        The maturity of the option, expressed in years (e.g. 0.25 for 3-month and 2 for 2 years)
    S: scalar or array_like
        The current price of the underlying asset.
    vol: scalar or array_like
        The implied Black-Scholes volatility.
    r: scalar or array_like
        The annualized risk-free interest rate, continuously compounded.
    q: scalar or array_like
        The annualized continuous dividend yield.
    callput: str
        Must be either 'call' or 'put'.

    Returns
    -------
    price: scalar or array_like
        The price of the option.

    Examples
    --------
    >>> blackscholes_price(95, 0.25, 100, 0.2, r=0.05, callput='put')
    1.5342604771222823
    """

    # Correction for zero maturity
    mask = T > 0
    price = np.maximum(S - K, 0) if callput.lower() == 'call' else np.maximum(K - S, 0)  # intrinsic value at maturity
    if np.any(mask): 
        F = S*np.exp((r-q)*T)
        v = np.sqrt(vol**2*T)
        with np.errstate(divide='ignore', invalid='ignore'):
            d1 = np.log(F/K)/v + 0.5*v
        d2 = d1 - v
        try:
            opttype = {'call':1, 'put':-1}[callput.lower()]
        except:
            raise ValueError('The value of callput must be either "call" or "put".')
        price = opttype*(F*norm.cdf(opttype*d1)-K*norm.cdf(opttype*d2))*np.exp(-r*T)
    return price

In [3]:
# Params
S0 = 100
vol = 0.2
r = 0.1
q = 0.02
K = 100

# Simulate paths
ts = np.linspace(0, 1, 13)
paths = blackscholes_mc(S=S0, vol=vol, r=r, q=q, ts=ts, npaths=100000)

### Case M_t = 0 ###
# Compute discounted payoff at each step along each path
discounted_payoff = np.maximum(K - paths, 0) * np.exp(-r * ts[:, np.newaxis])
# Compute option price as the mean of the maximum discounted payoff across time steps
upper_bound_zero = np.mean(np.max(discounted_payoff, axis=0))
print(f"Option upper price with M_t = 0: {upper_bound_zero:.4f}")

### Case M_t = BS_Put less initial value ###
BS_Put_start = blackscholes_price(K=K, T=1, S=S0, vol=vol, r=r, q=q, callput='put')
# Time to maturity at each time step
T_remaining = ts[-1] - ts
# Compute BS put prices along the paths
BS_Put_paths = np.exp(-r * ts[:,None]) * blackscholes_price(K=K, T=T_remaining, S=paths.T, vol=vol, r=r, q=q, callput='put').T - BS_Put_start
upper_bound_bs_put = np.mean(np.max(discounted_payoff - BS_Put_paths, axis=0))
print(f"Option upperprice with M_t = BS_Put less initial value: {upper_bound_bs_put:.4f}")

Option upper price with M_t = 0: 8.5991
Option upperprice with M_t = BS_Put less initial value: 5.3521


## Optimal martingale

The optimal martingale that gives zero <em>duality gap</em> is the martingale component of the discounted true value process (known as Snell envelope) $D_{0,t}V_t$, following the optimal exercise strategy.

To get a good upper bound, we first find a value process that is close to the true value process, and then extract its martingale component. This approximate value process can be obtained from a functional approximation such as regression, or from an exercise strategy (stopping time).

In general, for a stochastic process $\{U_n\}_{n\geq0}$, to extract its martingale component, we only need to let $M_0=0$, and then

$$M_{n+1}-M_n=U_{n+1}-\mathbb{E}\left[U_{n+1}\big|\mathcal{F}_n\right],\quad n=0,1,\cdots.$$

As a demonstration, we show below the code that calculates the upper bound estimate of the price of Bermudan put with monthly exercises as in Question 1.1. The optimal exercise frontier is given for each exercise date (from a PDE solver), i.e. if the underlying price falls below Bs[i] at time ts[i], then we should exercise the option, otherwise we should continue.

In [4]:
Bs = np.array([87.52900166,  87.80954309,  88.09098369,  88.37332635, 88.72752759,  89.1544435 ,  89.61925406,  90.19465661, 90.88274744,  91.75942312,  92.97867688,  94.85697075, 100.])

def exer_or_cont(i, S):
    return S <= Bs[i]

In [5]:
S0, K, vol, r, q = 100, 100, 0.2, 0.1, 0.02
ts = np.linspace(0, 1, 13)
exer_func = lambda S: np.maximum(K-S, 0)

# nested simulation from time t_i when stock price is S
def nested_mc(S, vol, r, q, i, ts, nnested):
    nested_paths = np.full(nnested, S, dtype=float)
    tot_payoff = 0
    for j in range(i+1, len(ts)):
        dt = ts[j] - ts[j-1]
        dW = np.random.randn(len(nested_paths))*np.sqrt(dt)        # Brownian increment
        nested_paths = nested_paths*np.exp((r-q)*dt)*np.exp(-0.5*vol**2*dt + vol*dW)
        exer_vals =  exer_func(nested_paths)
        if j < len(ts)-1:
            ind = exer_or_cont(j, nested_paths)                    # identify the paths that need exercise
            tot_payoff += np.sum(exer_vals[ind])*np.exp(-r*ts[j])
            nested_paths = nested_paths[~ind]                      # remove exercised paths 
            if len(nested_paths) == 0:                             # if exercised for all paths, stop
                break
        else:
            tot_payoff +=  np.sum(exer_vals)*np.exp(-r*ts[j])
    return tot_payoff/nnested                                      # taking average of paths

# Simulate independent paths and exercise the option acoording to this strategy
V0 = nested_mc(S0, vol, r, q, 0, ts, 10000000)
# Upper bound by Andersen-Broadie algorithm
npaths = 500                                                       # number of paths in the second independent run
nnested = 1000                                                     # number of paths used in nested simulation
paths = blackscholes_mc(S=S0, vol=vol, r=r, q=q, ts=ts, npaths=npaths)
V = np.full(paths.shape, np.nan, dtype=float)                   # discounted value process V_i (discounted to time zero)
EV = np.full(paths.shape, np.nan, dtype=float)                  # Conditional expectation E[V_{i+1}|F_i], or continuation value
V[0] = V0                                                          # Initial value from lower bound simulation
EV[0] = V0                                                         # at time 0, option value = continuation value
for i in range(1, len(ts)-1):
    exer_vals =  exer_func(paths[i])          
    ind = exer_or_cont(i, paths[i])                                # True for exercise False for continue
    for j in range(npaths):
        if exer_or_cont(i, paths[i][j]):
            V[i, j] = exer_vals[j]*np.exp(-r*ts[i])                     # if exercised, V[i,j] = exercise value
            EV[i,j] = nested_mc(paths[i, j], vol, r, q, i, ts, nnested) # launch nested simulation to estimate E[V_{i+1}|F_i]
        else:
            V[i,j] = nested_mc(paths[i, j], vol, r, q, i, ts, nnested)  # if continue, use nested simulation to estimate V[i, j]
            EV[i, j] = V[i, j]                                          # E[V_{i+1}|F_i] = V_i
V[-1] = exer_func(paths[-1])*np.exp(-r*ts[-1])                     # values at final maturity
hedges = np.zeros(paths.shape, dtype=float)
hedges[1:] = np.cumsum(V[1:]-EV[:-1], axis=0)                      # martingale increment V_{i+1}-E[V_{i+1}|F_i]

print('Primal Price (Lower Bound) = {:.4f}'.format(V0))
print('Dual Price   (Upper Bound) = {:.4f}'.format(np.mean(np.amax(exer_func(paths[1:])*np.exp(-r*ts[1:, np.newaxis])-hedges[1:], axis=0))))

Primal Price (Lower Bound) = 5.1511
Dual Price   (Upper Bound) = 5.1621


Since the strategy is optimal, we see that both lower bound and upper bound prices converge to the true value. 

However, in general, we may only get a sub-optimal strategy (e.g. from Longstaff-Schwartz algorithm). But with this sub-optimal strategy, we can build the (discounted) value process and then extract its martingale component to obtain an upper bound. Intuitively speaking, the closer this sub-optimal strategy is to the optimal one, the tighter the corresponding upper bound.

<b style="color:darkorange">Question 2.</b> Consider pricing the Bermudan put option as in Question 1 by using the constant $1.0$ and the Black-Scholes price of a European put option with volatility $0.2$ and maturity $T-t_n$ as the two basis functions at time $t_n$. Use the Longstaff-Schwartz algorithm to build an exercise strategy with these basis functions and then use the Broadie-Andersen algorithm to compute the corresponding upper bound.

For your convenience, an implementation of Longstaff-Schwartz algorithm is included in the cell below. It computes the regression coefficients at each exercise date.

In [6]:
S0, K, vol, r, q = 100, 100, 0.2, 0.1, 0.02
ts = np.linspace(0, 1, 13)


# Estimate the optimal exercise strategy using Longstaff-Schwartz algorithm
npaths = 1000000
paths = blackscholes_mc(S=S0, vol=vol, r=r, q=q, ts=ts, npaths=npaths)
betas_LS = np.full((len(ts), 2), np.nan, dtype=float)
payoff = np.maximum(K-paths[-1], 0)
for i in range(len(ts)-2, 0, -1):
    discount = np.exp(-r*(ts[i+1]-ts[i]))
    payoff = payoff*discount
    Z = blackscholes_price(K, ts[-1]-ts[i], paths[i], vol, r, q, callput='put')
    A = np.vstack((np.ones_like(Z), Z)).T
    betas_LS[i] = np.linalg.lstsq(A, payoff, rcond=None)[0]    # regression to estimate continuation values
    contval = betas_LS[i, 0]+betas_LS[i, 1]*Z
    exerval = np.maximum(K-paths[i], 0)
    # identify the paths where we should exercise
    ind = exerval > contval
    payoff[ind] = exerval[ind]                     # update payoff on exercised paths
betas_LS                                           # No regression needed at first and last time steps, return NaN

array([[            nan,             nan],
       [-4.72764941e-01,  1.29561595e+00],
       [-4.34283202e-01,  1.28121072e+00],
       [-3.63574456e-01,  1.25674557e+00],
       [-2.78071680e-01,  1.22631202e+00],
       [-1.98546232e-01,  1.19373537e+00],
       [-1.24316274e-01,  1.15949349e+00],
       [-7.23902658e-02,  1.12841317e+00],
       [-3.19144157e-02,  1.09791236e+00],
       [-6.13365705e-03,  1.06646811e+00],
       [ 1.23501080e-02,  1.03194348e+00],
       [ 1.13424693e-04,  9.99889388e-01],
       [            nan,             nan]])

The $\textsf{nested\_mc\_policy}$ function computes the option's value (lower bound) conditional on a specific exercise strategy provided as a parameter. It enables the generation of nested simulations starting from any arbitrary state $(t, S)$, applying the decision rules previously calibrated via the Longstaff-Schwartz algorithm.

In [7]:
def nested_mc_policy(S, vol, r, q, i, ts, nnested, policy):
    nested_paths = np.full(nnested, S, dtype=float)
    tot_payoff = 0
    for j in range(i+1, len(ts)):
        dt = ts[j] - ts[j-1]
        dW = np.random.randn(len(nested_paths))*np.sqrt(dt)        # Brownian increment
        nested_paths = nested_paths*np.exp((r-q)*dt)*np.exp(-0.5*vol**2*dt + vol*dW)
        exer_vals = np.maximum(K - nested_paths, 0)
        if j < len(ts)-1:
            Z = blackscholes_price(K, ts[-1]-ts[j], nested_paths, vol, r, q, callput='put')
            contval = policy[j, 0]+policy[j, 1]*Z 
            ind = exer_vals > contval                   # identify the paths that need exercise
            tot_payoff += np.sum(exer_vals[ind])*np.exp(-r*(ts[j]))
            nested_paths = nested_paths[~ind]                      # remove exercised paths 
            if len(nested_paths) == 0:                             # if exercised for all paths, stop
                break
        else:
            tot_payoff +=  np.sum(exer_vals)*np.exp(-r*(ts[j]))
    return tot_payoff/nnested                                      # taking average of paths

Apply Broadie-Andersen algorithm to compute upper bound

In [8]:
# Apply Broadie-Andersen algorithm to compute upper bound
npaths = 500                                                       # number of paths in the second independent run
nnested = 1000                                                    # number of paths used in nested simulation
paths = blackscholes_mc(S=S0, vol=vol, r=r, q=q, ts=ts, npaths=npaths) # Simulate independent paths
V = np.full(paths.shape, np.nan, dtype=float)                   # discounted value process V_i (discounted to time zero)
EV = np.full(paths.shape, np.nan, dtype=float)                  # Conditional expectation E[V_{i+1}|F_i], or continuation value

V0 = nested_mc_policy(S0, vol, r, q, 0, ts, 1000000, betas_LS) # We compute V0 using the nested_mc_policy function alone to ensure consistency with high number of paths
V[0] = V0                                                          # Initial value from lower bound simulation
EV[0] = V0                                                         # at time 0, option value = continuation value

for i in range(1, len(ts)-1):
    # Compute exercise and continuation values along all paths at time t_i using the exercise strategy from LS
    exer_vals =  np.maximum(K - paths[i], 0)
    Z = blackscholes_price(K, ts[-1]-ts[i], paths[i], vol, r, q, callput='put')
    contval = betas_LS[i, 0]+betas_LS[i, 1]*Z
    ind = exer_vals > contval                                # True for exercise False for continue

    # Compute V_i and E[V_{i+1}|F_i] using nested simulation
    for j in range(npaths):
        if ind[j]:
            V[i, j] = exer_vals[j]*np.exp(-r*ts[i])                     # if exercised, V[i,j] = exercise value
            EV[i,j] = nested_mc_policy(paths[i, j], vol, r, q, i, ts, nnested, betas_LS) # launch nested simulation to estimate E[V_{i+1}|F_i]
        else:
            V[i,j] = nested_mc_policy(paths[i, j], vol, r, q, i, ts, nnested, betas_LS)  # if continue, use nested simulation to estimate V[i, j]
            EV[i, j] = V[i, j]                                          # E[V_{i+1}|F_i] = V_i

V[-1] = np.maximum(K - paths[-1], 0)*np.exp(-r*ts[-1])                     # values at final maturity

# Compute hedging martingale
hedges = np.zeros(paths.shape, dtype=float)
hedges[1:] = np.cumsum(V[1:]-EV[:-1], axis=0)                      # martingale increment V_{i+1}-E[V_{i+1}|F_i]
print('Primal Price (Lower Bound) = {:.4f}'.format(V0))
print('Dual Price   (Upper Bound) = {:.4f}'.format(np.mean(np.amax(np.maximum(K - paths[1:], 0)*np.exp(-r*ts[1:, np.newaxis])-hedges[1:], axis=0))))

Primal Price (Lower Bound) = 5.1478
Dual Price   (Upper Bound) = 5.1719


<b style="color:darkorange">Question 3.</b> <b>(An alternative algorithm to estimate the upper bound)</b> The primal algorithm (Longstaff-Schwartz or Tsitsiklis-Van Roy) provides a function approximation (via regression) of the continuation value $C_n$ at $t_n$, hence of the value function $V_n = \max(C_n,F_n)$ where $F_n$ is the exercise value. We can use this functional approximation of the value function to write an alternative algorithm in order to estimate the upper bound: at each time $t_n$, use the functional approximation of $V_n$, and run the nested simulation to estimate the conditional expectation $\mathbb{E}\left[V_{n+1}\big|\mathcal{F}_n\right]$ (using the functional approximation of $V_{n+1}$). Note that this nested simulation only needs to be run for one time step.

For the same Bermudan put option and same basis functions, use TVR algorithm to build the value process and then extract its martingale component using the alternative algorithm explained above to estimate the upper bound price.

For your convenience, an implementation of TVR algorithm is included in the cell below. It computes the regression coefficients at each exercise date. 

In [9]:
S0, K, vol, r, q = 100, 100, 0.2, 0.1, 0.02
ts = np.linspace(0, 1, 13)

npaths = 1000000
paths = blackscholes_mc(S=S0, vol=vol, r=r, q=q, ts=ts, npaths=npaths)
betas_TVR = np.full((len(ts), 2), np.nan, dtype=float)
V = np.maximum(K-paths[-1], 0)
for i in range(len(ts)-2, 0, -1):
    discount = np.exp(-r*(ts[i+1]-ts[i]))
    Z = blackscholes_price(K, ts[-1]-ts[i], paths[i], vol, r, q, callput='put')
    A = np.vstack((np.ones_like(Z), Z)).T
    betas_TVR[i] = np.linalg.lstsq(A, V*discount, rcond=None)[0]      # regression to estimate continuation values
    contval = betas_TVR[i, 0]+betas_TVR[i, 1]*Z
    exerval = np.maximum(K-paths[i], 0)
    V = np.maximum(exerval, contval)                      # compute values
betas_TVR                                                 # No regression needed at first and last time steps, return NaN

# nested simulation from time t_i when stock price is S using exercise policy for 1 time step
def nested_mc_1_step(S, vol, r, q, i, ts, nnested, policy):
    dt = ts[i+1] - ts[i]
    dW = np.random.randn(nnested)*np.sqrt(dt)       
    S_next = S*np.exp((r-q)*dt)*np.exp(-0.5*vol**2*dt + vol*dW)
    exer_vals = np.maximum(K - S_next, 0)

    # Determine V_{i+1} based on exercise policy
    if i+1 == len(ts)-1:                             
        v_next  = exer_vals
    else: 
        Z = blackscholes_price(K, ts[-1]-ts[i+1], S_next, vol, r, q, callput='put')
        contval = policy[i+1, 0]+policy[i+1, 1]*Z 
        v_next = np.maximum(exer_vals, contval)           
    
    return np.mean(v_next)                                     # taking average of paths


npaths = 1000                                                    # number of paths in the second independent run
nnested = 1000                                                    # number of paths used in nested simulation
paths = blackscholes_mc(S=S0, vol=vol, r=r, q=q, ts=ts, npaths=npaths) # Simulate independent paths
V = np.full(paths.shape, np.nan, dtype=float)                   # discounted value process V_i (discounted to time zero)
EV = np.full(paths.shape, np.nan, dtype=float)                  # Conditional expectation E[V_{i+1}|F_i], or continuation value
hedges = np.zeros(paths.shape, dtype=float)                     # hedging martingale
hedges[0] = 0                                                   # at time 0, hedging martingale is zero

for i in range(len(ts)-1):
    # Compute exercise and continuation values along all paths at time t_i using the exercise strategy from TVR
    exer_vals =  np.maximum(K - paths[i+1], 0)

    # Determine V_i based on exercise policy
    if i + 1 == len(ts)-1:                     
        V[i] = exer_vals
    else:
        Z = blackscholes_price(K, ts[-1]-ts[i+1], paths[i+1], vol, r, q, callput='put')
        contval = betas_TVR[i+1, 0]+betas_TVR[i+1, 1]*Z
        V[i] = np.maximum(exer_vals, contval)                              

    # Compute EV_i using nested simulation
    for j in range(npaths):
        EV[i, j] = nested_mc_1_step(paths[i, j], vol, r, q, i, ts, nnested, betas_TVR)

    # Update hedging martingale
    hedges[i+1] = hedges[i] + (V[i]-EV[i])*np.exp(-r*ts[i+1])

print('Dual Price (Upper Bound) = {:.4f}'.format(np.mean(np.amax(np.maximum(K - paths, 0)*np.exp(-r*ts[:, np.newaxis])-hedges, axis=0))))        

Dual Price (Upper Bound) = 5.1717


The upper bound estimator can be used to check if the basis functions used in regression are adequate. A tight duality gap between the lower bound and upper bound is a confirmation that the linear span of the selected basis functions indeed gives a good approximation to the continuation value. We now illustrate this with the pricing of Bermudan-Asian call options.

<b style="color:darkorange">Question 4</b> (<b>Bermudan-Asian Call Option</b>). In Question 2.4 of Homework I, we priced a Bermudan-Asian call option using a particular set of basis functions. Assume that all the parameters stay the same.

<b>(a).</b> Reuse the exercise strategy derived from the basis functions used in Question 2.4 of Homework I by Longstaff-Schwartz with $\bar{\sigma}=0.1$ to estimate the lower and upper bound. For your convenience, an implementation of Longstaff-Schwartz is included in the cell below. Note that the lower bound should be obtained by running new independent paths stopped according to the strategy from the Longstaff-Schwartz algorithm.

In [10]:
# Longstaff-Schwartz
S0, K, vol, r, q = 100, 100, 0.2, 0, 0
ts = np.linspace(0, 1, 13)
npaths = 1000000
pathsS = blackscholes_mc(S0, vol, r, q, ts=ts, npaths=npaths) 
vol_ = 0.1                                 # stock prices
pathsA = np.vstack((pathsS[0], np.cumsum(pathsS[1:], axis=0)/np.arange(1, 13)[:, np.newaxis])) # running averages
payoff = np.maximum(pathsA[-1]-K, 0)
betas_AC = np.full((len(ts), 2), np.nan, dtype=float)

for i in range(len(ts)-2, 0, -1):
    discount = np.exp(-r*(ts[i+1]-ts[i]))
    payoff = payoff*discount
    Z = (i*pathsA[i]+(12-i)*pathsS[i])/12
    callZ = blackscholes_price(K, ts[-1]-ts[i], Z, vol_, callput='call')
    A = np.vstack((np.ones_like(Z), callZ)).T
    betas_AC[i] = np.linalg.lstsq(A, payoff, rcond=None)[0]           # regression to estimate continuation values
    contval = betas_AC[i, 0]+betas_AC[i, 1]*callZ
    exerval = np.maximum(pathsA[i]-K, 0)
    ind = exerval > contval                               # identify the paths where we should exercise
    payoff[ind] = exerval[ind]
betas_AC # no need to estimate continuation values at the first date (not an exercise date) or the last date (always exercise)

array([[        nan,         nan],
       [ 0.25876366,  1.13194592],
       [-0.02034759,  1.11037295],
       [-0.19334074,  1.09048417],
       [-0.30000641,  1.07416028],
       [-0.35532917,  1.06071911],
       [-0.38374784,  1.05049945],
       [-0.38587636,  1.04163656],
       [-0.36356923,  1.03318004],
       [-0.31907072,  1.02536595],
       [-0.25553063,  1.01722012],
       [-0.16956619,  1.00784908],
       [        nan,         nan]])

Define nested_mc_policy_asian to compute nested monte-carlo for the pricing of the asian option

In [11]:
def nested_mc_policy_asian(S,A, K, vol_sim, vol_proxy, r, q, i, ts, nnested, policy):
    nested_paths = np.full(nnested, S, dtype=float)
    nested_A = np.full(nnested, A, dtype=float)
    tot_payoff = 0
    
    for j in range(i+1, len(ts)):
        dt = ts[j] - ts[j-1]
        dW = np.random.randn(len(nested_paths))*np.sqrt(dt)        # Brownian increment
        nested_paths = nested_paths*np.exp((r-q)*dt)*np.exp(-0.5*vol_sim**2*dt + vol_sim*dW)
        nested_A = ((j-1)*nested_A + nested_paths ) / j
        exer_vals = np.maximum(nested_A - K, 0)
        if j < len(ts)-1:
            Z = (j*nested_A+(12-j)*nested_paths)/12
            callZ = blackscholes_price(K, ts[-1]-ts[j], Z, vol_proxy, callput='call')
            contval = policy[j, 0]+policy[j, 1]*callZ
            ind = exer_vals > contval                   # identify the paths that need exercise
            tot_payoff += np.sum(exer_vals[ind])*np.exp(-r * ts[j])
            nested_paths = nested_paths[~ind]
            nested_A = nested_A[~ind]                      # remove exercised paths
        else:
            tot_payoff +=  np.sum(exer_vals)*np.exp(-r * ts[j])
    return tot_payoff/nnested                                      # taking average of paths

Compute the upper and lower bound

In [12]:
# Apply Broadie-Andersen algorithm to compute upper bound
npaths = 1000
pathsS = blackscholes_mc(S0, vol, r, q, ts=ts, npaths=npaths)                                  # stock prices
pathsA = np.vstack((pathsS[0], np.cumsum(pathsS[1:], axis=0)/np.arange(1, 13)[:, np.newaxis])) # running averages
payoff = np.maximum(pathsA[-1]-K, 0)              

# V0 provide lower bound
V0 = nested_mc_policy_asian(S0, S0, K, vol, vol_, r, q, 0, ts, 1000000, betas_AC)
V = np.full(pathsS.shape, np.nan, dtype=float)                   # discounted value process V_i (discounted to time zero)
EV = np.full(pathsS.shape, np.nan, dtype=float)                  # Conditional expectation E
V[0] = V0                                                          # Initial value from lower bound simulation
EV[0] = V0                                                         # at time 0, option value = continuation value

nnested = 1000                                                 # number of paths used in nested simulation

for i in range(1, len(ts)-1):
    # Compute exercise and continuation values along all paths at time t_i using the exercise strategy from LS
    exer_vals =  np.maximum(pathsA[i] - K, 0)
    Z = (i * pathsA[i] + (12 - i) * pathsS[i]) / 12
    Call_z = blackscholes_price(K, ts[-1]-ts[i], Z, vol_, r, q, callput='call')
    contval = betas_AC[i, 0]+betas_AC[i, 1]*Call_z
    ind = exer_vals > contval                                # True for exercise False for continue

    # Compute V_i and E[V_{i+1}|F_i] using nested simulation
    for j in range(npaths):
        if ind[j]:
            V[i, j] = exer_vals[j]*np.exp(-r*ts[i])                     # if exercised, V[i,j] = exercise value
            EV[i,j] = nested_mc_policy_asian(pathsS[i, j],pathsA[i, j], K, vol, vol_, r, q, i, ts, nnested, betas_AC)*np.exp(-r * ts[i]) # launch nested simulation to estimate E[V_{i+1}|F_i]
        else:
            V[i,j] = nested_mc_policy_asian(pathsS[i, j],pathsA[i, j], K, vol, vol_, r, q, i, ts, nnested, betas_AC)*np.exp(-r * ts[i])  # if continue, use nested simulation to estimate V[i, j]
            EV[i, j] = V[i, j]                                         # E[V_{i+1}|F_i] = V_i

V[-1] = np.maximum(pathsA[-1]-K, 0)*np.exp(-r*ts[-1])                     # values at final maturity

# Compute hedging martingale
hedges = np.zeros(pathsS.shape, dtype=float)
hedges[1:] = np.cumsum(V[1:]-EV[:-1], axis=0)   

upper_bound = np.mean(np.amax(np.maximum(pathsA[1:]-K, 0)*np.exp(-r*ts[1:, np.newaxis])-hedges[1:], axis=0))
print('Primal Price (Lower Bound) = {:.4f}'.format(V0))
print('Dual Price   (Upper Bound) = {:.4f}'.format(upper_bound))
print('Gap between Lower and Upper Bound = {:.4f}'.format(upper_bound - V0))

Primal Price (Lower Bound) = 5.3159
Dual Price   (Upper Bound) = 5.4177
Gap between Lower and Upper Bound = 0.1018


<b>4 (b).</b> As an alternative set of basis functions at time $t_n$, use 
$$1, A_{t_n}, S_{t_n}, A_{t_n}^2, S_{t_n}^2, A_{t_n}S_{t_n}.$$
in the Longstaff-Schwartz algorithm and then give the corresponding lower bound and upper bound.

In [13]:
# Longstaff-Schwartz
S0, K, vol, r, q = 100, 100, 0.2, 0, 0
ts = np.linspace(0, 1, 13)
npaths = 1000000
pathsS = blackscholes_mc(S0, vol, r, q, ts=ts, npaths=npaths) 
vol_ = 0.1                                 # stock prices
pathsA = np.vstack((pathsS[0], np.cumsum(pathsS[1:], axis=0)/np.arange(1, 13)[:, np.newaxis])) # running averages
payoff = np.maximum(pathsA[-1]-K, 0)
betas_AC_2 = np.full((len(ts), 6), np.nan, dtype=float)

for i in range(len(ts)-2, 0, -1):
    discount = np.exp(-r*(ts[i+1]-ts[i]))
    payoff = payoff*discount
    A = np.vstack((np.ones_like(pathsA[i]),pathsS[i], pathsA[i],pathsS[i]*pathsS[i],pathsA[i]*pathsA[i],pathsA[i]*pathsS[i])).T
    betas_AC_2[i] = np.linalg.lstsq(A, payoff, rcond=None)[0]           # regression to estimate continuation values
    contval = (betas_AC_2[i, 0] + betas_AC_2[i, 1]*pathsS[i] + betas_AC_2[i, 2]*pathsA[i] +
               betas_AC_2[i, 3]*pathsS[i]*pathsS[i] + betas_AC_2[i, 4]*pathsA[i]*pathsA[i] +
               betas_AC_2[i, 5]*pathsA[i]*pathsS[i])
    exerval = np.maximum(pathsA[i]-K, 0)
    ind = exerval > contval                               # identify the paths where we should exercise
    payoff[ind] = exerval[ind]

In [14]:
def nested_mc_policy_asian_2(S,A, K, vol_sim, vol_proxy, r, q, i, ts, nnested, policy):
    nested_paths = np.full(nnested, S, dtype=float)
    nested_A = np.full(nnested, A, dtype=float)
    tot_payoff = 0
    
    for j in range(i+1, len(ts)):
        dt = ts[j] - ts[j-1]
        dW = np.random.randn(len(nested_paths))*np.sqrt(dt)        # Brownian increment
        nested_paths = nested_paths*np.exp((r-q)*dt)*np.exp(-0.5*vol_sim**2*dt + vol_sim*dW)
        nested_A = ((j-1)*nested_A + nested_paths ) / j
        exer_vals = np.maximum(nested_A - K, 0)
        if j < len(ts)-1:
            contval = (policy[j, 0]+policy[j, 1]*nested_paths + policy[j, 2]*nested_A + 
                      policy[j, 3]*nested_paths*nested_paths + policy[j, 4]*nested_A*nested_A +
                    policy[j,5]*nested_A*nested_paths)
            ind = exer_vals > contval                   # identify the paths that need exercise
            tot_payoff += np.sum(exer_vals[ind])*np.exp(-r * ts[j])
            nested_paths = nested_paths[~ind]
            nested_A = nested_A[~ind]                      # remove exercised paths
        else:
            tot_payoff +=  np.sum(exer_vals)*np.exp(-r * ts[j])
    return tot_payoff/nnested                                      # taking average of paths

In [15]:
# Apply Broadie-Andersen algorithm to compute upper bound
npaths = 1000
pathsS = blackscholes_mc(S0, vol, r, q, ts=ts, npaths=npaths)                                  # stock prices
pathsA = np.vstack((pathsS[0], np.cumsum(pathsS[1:], axis=0)/np.arange(1, 13)[:, np.newaxis])) # running averages
payoff = np.maximum(pathsA[-1]-K, 0)              

V0 = nested_mc_policy_asian_2(S0, S0, K, vol, vol_, r, q, 0, ts, 1000000, betas_AC_2)
V = np.full(pathsS.shape, np.nan, dtype=float)                   # discounted value process V_i (discounted to time zero)
EV = np.full(pathsS.shape, np.nan, dtype=float)                  # Conditional expectation E
V[0] = V0                                                          # Initial value from lower bound simulation
EV[0] = V0                                                         # at time 0, option value = continuation value

nnested = 10000                                                 # number of paths used in nested simulation

for i in range(1, len(ts)-1):
    # Compute exercise and continuation values along all paths at time t_i using the exercise strategy from LS
    exer_vals =  np.maximum(pathsA[i] - K, 0)
    contval = (betas_AC_2[i, 0] + betas_AC_2[i, 1]*pathsS[i] + betas_AC_2[i, 2]*pathsA[i] +
               betas_AC_2[i, 3]*pathsS[i]*pathsS[i] + betas_AC_2[i, 4]*pathsA[i]*pathsA[i] + 
               betas_AC_2[i, 5]*pathsA[i]*pathsS[i])
    ind = exer_vals > contval                                # True for exercise False for continue

    # Compute V_i and E[V_{i+1}|F_i] using nested simulation
    for j in range(npaths):
        if ind[j]:
            V[i, j] = exer_vals[j]*np.exp(-r*ts[i])                     # if exercised, V[i,j] = exercise value
            EV[i,j] = nested_mc_policy_asian_2(pathsS[i, j],pathsA[i, j], K, vol, vol_, r, q, i, ts, nnested, betas_AC_2)*np.exp(-r * ts[i]) # launch nested simulation to estimate E[V_{i+1}|F_i]
        else:
            V[i,j] = nested_mc_policy_asian_2(pathsS[i, j],pathsA[i, j], K, vol, vol_, r, q, i, ts, nnested, betas_AC_2)*np.exp(-r * ts[i])  # if continue, use nested simulation to estimate V[i, j]
            EV[i, j] = V[i, j]                                         # E[V_{i+1}|F_i] = V_i

V[-1] = np.maximum(pathsA[-1]-K, 0)*np.exp(-r*ts[-1])                     # values at final maturity

# Compute hedging martingale
hedges = np.zeros(pathsS.shape, dtype=float)
hedges[1:] = np.cumsum(V[1:]-EV[:-1], axis=0)                      # martingale increment V_{i+1}-E[V_{i+1}|F_i]

upper_bound = np.mean(np.amax(np.maximum(pathsA[1:]-K, 0)*np.exp(-r*ts[1:, np.newaxis])-hedges[1:], axis=0))
print('Primal Price (Lower Bound) = {:.4f}'.format(V0))
print('Dual Price   (Upper Bound) = {:.4f}'.format(upper_bound))
print('Gap between Lower and Upper Bound = {:.4f}'.format(upper_bound - V0))

Primal Price (Lower Bound) = 5.0997
Dual Price   (Upper Bound) = 5.4515
Gap between Lower and Upper Bound = 0.3519


<b>(c).</b> Compare the numerical duality gaps obtained from (a) and (b). Which set of basis functions does a better job at estimating the optimal exercise strategy?

**Answer:**

The duality gap is much smaller for set (a) (≈ 0.1) than for set (b) (≈ 0.3), which indicates that set (a) provides a better approximation of the optimal exercise strategy.This is mainly because the Black–Scholes proxy already captures key financial features of the option value, such as convexity and the correct asymptotic behavior. 
The regression therefore only needs to learn a correction to this proxy, which is relatively easy.

In contrast, the polynomial basis in set (b) has no financial structure. It can behave poorly for large values of $S$ or $A$ and does not naturally capture the exercise boundary, making the approximation less accurate.

Overall, set (a) is clearly more efficient and leads to a more reliable exercise strategy.
